In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import guidance
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'# pathlib.Path(__file__).parent.resolve()
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")

with open(os.path.expanduser('~/.bing_api_key'), 'r') as file:
    subscription_key = file.read().replace('\n', '')

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    # search_url =
#     search_term = "Who are the founders of OpenAI?"

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            # print("SEARCHING BING")
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        # print(type(_bing_cache[params_key]))
        # print(list(_bing_cache[params_key]))
        # print(_bing_cache[params_key])
        # result_batch = _bing_cache[params_key]["webPages"]["value"]

        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])

        search_results.extend(data)

    return search_results


In [3]:
def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]

In [4]:
llm = guidance.llms.OpenAI("gpt-4", chat_completion=True, temperature=0) # could also specify default temperature here


In [5]:
top_snippets("OpenAI founders")

[{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'title': 'Sam Altman - Wikipedia',
  'snippet': 'Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former presi

In [10]:
import re
def is_search(completion):
    return '<search>' in completion
def search(query):
    query = re.search(r'<search>(.*)', query).group(1)
    # SEARCH\((.*)\)', query).group(1)
    return top_snippets(query, n=3)
prompt = guidance('''<|im_start|>system
Assistant is a large language model trained by OpenAI.
Whenever its response depends on any factual information, it searches the web by using the function <search>query</search> before responding to the user.
For example, it might start its response with
<search>query</search>
After which the top snippets from search will be displayed.<|im_end|>{{#each history}}
<|im_start|>{{this.user}}
{{this.utterance}}<|im_end|>{{/each}}
<|im_start|>user
{{query}}<|im_end|>
<|im_start|>assistant
{{generate "answer" stop="</search>"}}</search>
{{#if (is_search answer)}}{{#each (search answer)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}<|im_end|>
<|im_start|>user
Given the search results above, please answer my question<|im_end|>
{{generate "more"}}
{{/if}}''', llm=llm)
# query = "What is Facebook's stock price?"
query = "Where does Scott Lundberg Work?"
a = prompt(is_search=is_search, search=search, query=query,
       history=[
    {'user': 'user', 'utterance': 'Who are the founders of OpenAI?'}, 
    {'user': 'assistant', 'utterance': '''<search>(OpenAI founders)</search>
<result>OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is a American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result>''' },
    {'user': 'user', 'utterance': '''Given the search results above, please answer my question'''},
    {'user': 'assistant', 'utterance': '''The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others'''}
    ])
a


{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'stop': '</search>', 'messages': [{'role': 'system', 'content': 'Assistant is a large language model trained by OpenAI.\nWhenever its response depends on any factual information, it searches the web by using the function <search>query</search> before responding to the user.\nFor example, it might start its response with\n<search>query</search>\nAfter which the top snippets from search will be displayed.'}, {'role': 'user', 'content': 'Who are the founders of OpenAI?'}, {'role': 'assistant', 'content': '<search>(OpenAI founders)</search>\n<result>OpenAI - Wikipedia\nOpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a dono

<|im_start|>system
Assistant is a large language model trained by OpenAI.
Whenever its response depends on any factual information, it searches the web by using the function <search>query</search> before responding to the user.
For example, it might start its response with
<search>query</search>
After which the top snippets from search will be displayed.<|im_end|>{{!--GMARKER_START_each${{#each history}}
<|im_start|>{{this.user}}
{{this.utterance}}<|im_end|>{{/each}}$--}}{{!--GMARKER_each$$--}}
<|im_start|>{{!--GMARKER_START_variable_ref$&#123;&#123;this.user&#125;&#125;$--}}user{{!--GMARKER_END_variable_ref$$--}}
{{!--GMARKER_START_variable_ref$&#123;&#123;this.utterance&#125;&#125;$--}}Who are the founders of OpenAI?{{!--GMARKER_END_variable_ref$$--}}<|im_end|>{{!--GMARKER_each$$--}}
<|im_start|>{{!--GMARKER_START_variable_ref$&#123;&#123;this.user&#125;&#125;$--}}assistant{{!--GMARKER_END_variable_ref$$--}}
{{!--GMARKER_START_variable_ref$&#123;&#123;this.utterance&#125;&#125;$--}}<search>(OpenAI founders)</search>
<result>OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is a American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result>{{!--GMARKER_END_variable_ref$$--}}<|im_end|>{{!--GMARKER_each$$--}}
<|im_start|>{{!--GMARKER_START_variable_ref$&#123;&#123;this.user&#125;&#125;$--}}user{{!--GMARKER_END_variable_ref$$--}}
{{!--GMARKER_START_variable_ref$&#123;&#123;this.utterance&#125;&#125;$--}}Given the search results above, please answer my question{{!--GMARKER_END_variable_ref$$--}}<|im_end|>{{!--GMARKER_each$$--}}
<|im_start|>{{!--GMARKER_START_variable_ref$&#123;&#123;this.user&#125;&#125;$--}}assistant{{!--GMARKER_END_variable_ref$$--}}
{{!--GMARKER_START_variable_ref$&#123;&#123;this.utterance&#125;&#125;$--}}The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others{{!--GMARKER_END_variable_ref$$--}}<|im_end|>{{!--GMARKER_each$$--}}{{!--GMARKER_END_each$$--}}
<|im_start|>user
{{!--GMARKER_START_variable_ref$&#123;&#123;query&#125;&#125;$--}}Where does Scott Lundberg Work?{{!--GMARKER_END_variable_ref$$--}}<|im_end|>
<|im_start|>assistant
{{!--GMARKER_START_generate$&#123;&#123;generate "answer" stop="</search>"&#125;&#125;$--}}<search>(Scott Lundberg work){{!--GMARKER_END_generate$$--}}</search>
{{!--GMARKER_START_if${{#if (is_search answer)}}{{#each (search answer)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}<|im_end|>
<|im_start|>user
Given the search results above, please answer my question<|im_end|>
{{generate "more"}}
{{/if}}$--}}{{!--GMARKER_START_each${{#each (search answer)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}$--}}{{!--GMARKER_each$$--}}
<result>
{{!--GMARKER_START_variable_ref$&#123;&#123;this.title&#125;&#125;$--}}Scott Lundberg{{!--GMARKER_END_variable_ref$$--}}
{{!--GMARKER_START_variable_ref$&#123;&#123;this.snippet&#125;&#125;$--}}Scott Lundberg Senior Researcher Microsoft Research. Biography. I am a Senior Researcher at Mi

In [14]:
messages = {'model': 'gpt-4', 'max_tokens': 500, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'stop': '\n', 'messages': [{'role': 'system', 'content': 'Assistant is a large language model trained by OpenAI.\nWhenever its response depends on any factual information, it searches the web by using the function <search>query</search> before responding to the user.\nFor example, it might start its response with\nSEARCH(query)\nAfter which the top snippets from search will be displayed.'}, {'role': 'user', 'content': 'Who are the founders of OpenAI?'}, {'role': 'assistant', 'content': 'SEARCH(OpenAI founders)'}, {'role': 'assistant', 'content': '<result>OpenAI - Wikipedia\nOpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.\n</result>\n<result>About - OpenAI\nAbout OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI\n</result>\n<result>Sam Altman - Wikipedia\nSamuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]\n</result>'}, {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others'}, {'role': 'user', 'content': "What is Facebook's stock price?"}, {'role': 'assistant', 'content': 'SEARCH(Facebook stock price)'}, {'role': 'assistant', 'content': "[{'title': 'Meta Platforms, Inc. (META) Stock Price, News, Quote &amp; History - Yahoo!', 'snippet': 'Meta Platforms, Inc. (META) Stock Price, News, Quote & History - Yahoo Finance U.S. markets open in 2 hours 26 minutes Dow Futures Meta Platforms, Inc. (META) NasdaqGS - NasdaqGS Real Time...'}, {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider', 'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'}, {'title': 'Meta Platforms Inc (FB) Stock Price &amp; News - Google Finance', 'snippet': '$159.28 AAPL1.19% Amazon.com, Inc. $100.61 AMZN2.97% NVIDIA Corporation $261.99 NVDA1.15% Tesla Inc $197.58 TSLA7.82% Alphabet Inc Class C $105.84 GOOG3.84% Alphabet Inc Class A $104.92 GOOGL3.66%...'}]\n<result>\nMeta Platforms, Inc. (META) Stock Price, News, Quote &amp; History - Yahoo!\nMeta Platforms, Inc. (META) Stock Price, News, Quote & History - Yahoo Finance U.S. markets open in 2 hours 26 minutes Dow Futures Meta Platforms, Inc. (META) NasdaqGS - NasdaqGS Real Time...\n</result>\n<result>\nNews | FACEBOOK Stock Price Today | Analyst Opinions - Insider\nStock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...\n</result>\n<result>\nMeta Platforms Inc (FB) Stock Price &amp; News - Google Finance\n$159.28 AAPL1.19% Amazon.com, Inc. $100.61 AMZN2.97% NVIDIA Corporation $261.99 NVDA1.15% Tesla Inc $197.58 TSLA7.82% Alphabet Inc Class C $105.84 GOOG3.84% Alphabet Inc Class A $104.92 GOOGL3.66%...\n</result>"}]}
messages
import openai
openai.ChatCompletion.create(**messages)

<OpenAIObject chat.completion id=chatcmpl-6xMUkNHlVKDpTmCt9Kw7UNYjrJE0B at 0x7f21a12eb450> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I am unable to provide real-time stock prices. However, you can check the current stock price of Meta Platforms, Inc. (formerly known as Facebook) by searching for the ticker symbol \"META\" on a financial news website or using a stock market app.",
        "role": "assistant"
      }
    }
  ],
  "created": 1679605842,
  "id": "chatcmpl-6xMUkNHlVKDpTmCt9Kw7UNYjrJE0B",
  "model": "gpt-4-0314",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 52,
    "prompt_tokens": 950,
    "total_tokens": 1002
  }
}